## Part B- Event Producer 2 

Name : Ko Ko Win <br> 
ID: 31842305

In [12]:
import pandas as pd 
from time import sleep 
from json import dumps 
from kafka import KafkaProducer 
import random 
import datetime as dt 
import pymongo 
from pymongo import MongoClient 
from pprint import pprint
import json

def read_data(): 
    aqua_streaming = pd.read_csv("hotspot_AQUA_streaming.csv")
    
    aqua_array = []
    
    for index, row in aqua_streaming.iterrows(): 
        aq_docs = {}
        aq_docs["latitude"] = float(row["latitude"])
        aq_docs["longitude"] = float(row["longitude"])
        aq_docs["confidence"] = int(row["confidence"]) 
        aq_docs["surface_temperature_celcius"] = int(row["surface_temperature_celcius"])
        
        aqua_array.append(aq_docs)
    
    return aqua_array 

def publish_message(producer_instance, topic_name, key, data):
    try:
        str_value = json.dumps(data, indent =3) #convert to JSON object
        key_bytes = bytes(key, encoding="utf-8") #cast key to bytes
        value_bytes = bytes(str_value, encoding="utf-8") #cast value to bytes 
        #send key-value bytes to kafka topic (topic_name)
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str_value) 
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    


if __name__ == '__main__':
    topic = "Producer02" 
    print('Publishing records..')
    producer = connect_kafka_producer()
    date = dt.datetime.now() #get current real world datetime 
    date += dt.timedelta(hours = 4.8) #plus 4.8 hours because data is send every 2 secs
    aq_data = read_data()
    
    while True: 
        random_num = random.randrange(0, len(aq_data)) #generate random integer
        data = aq_data[random_num] #access the climate streaming based on the random_num 
        data["created_time"] = date.strftime("%H:%M:%S") #grab the time and append it in the data
        data["producer_information"] = "Event Producer 2 (Aqua)" #append producer info
        publish_message(producer, topic, 'parsed', data) 
        date += dt.timedelta(hours = 4.8) #increment 4.8 hours for next batch 
        
        sleep(2)
        

Publishing records..
Message published successfully. Data: {
   "latitude": -34.2695,
   "longitude": 142.2025,
   "confidence": 80,
   "surface_temperature_celcius": 54,
   "created_time": "01:20:16",
   "producer_information": "Event Producer 2 (Aqua)"
}
Message published successfully. Data: {
   "latitude": -38.0423,
   "longitude": 146.4048,
   "confidence": 84,
   "surface_temperature_celcius": 66,
   "created_time": "06:08:16",
   "producer_information": "Event Producer 2 (Aqua)"
}
Message published successfully. Data: {
   "latitude": -37.7529,
   "longitude": 143.3745,
   "confidence": 85,
   "surface_temperature_celcius": 59,
   "created_time": "10:56:16",
   "producer_information": "Event Producer 2 (Aqua)"
}
Message published successfully. Data: {
   "latitude": -36.7783,
   "longitude": 146.6449,
   "confidence": 70,
   "surface_temperature_celcius": 56,
   "created_time": "15:44:16",
   "producer_information": "Event Producer 2 (Aqua)"
}
Message published successfully. Dat

KeyboardInterrupt: 